# Trie-Based Stemming Analysis

This notebook explores stemming using prefix and suffix tries. We analyze a list of nouns from `brown_nouns.txt` and identify stem + suffix pairs based on branching heuristics.

In [1]:
# Importing necessary libraries and Trie classes
from collections import defaultdict

class TrieNode:
    def __init__(self):
        self.children = defaultdict(TrieNode)
        self.count = 0

class Trie:
    def __init__(self, reverse=False):
        self.root = TrieNode()
        self.reverse = reverse

    def insert(self, word):
        if self.reverse:
            word = word[::-1]
        node = self.root
        for char in word:
            node = node.children[char]
            node.count += 1

    def find_split(self, word):
        if self.reverse:
            word = word[::-1]
        node = self.root
        max_branch = 0
        split_index = 0
        for i, char in enumerate(word):
            node = node.children[char]
            branch_count = len(node.children)
            if branch_count > max_branch:
                max_branch = branch_count
                split_index = i + 1
        if self.reverse:
            stem = word[:split_index][::-1]
            suffix = word[split_index:][::-1]
        else:
            stem = word[:split_index]
            suffix = word[split_index:]
        return stem, suffix

## Loading Words and Analyzing Tries

We load the nouns from the text file and analyze them using both prefix and suffix tries.

In [2]:
# Loading and analyzing
def load_words(filepath):
    with open(filepath, 'r') as f:
        words = [line.strip().lower() for line in f if line.strip()]
    return words

def analyze_trie(words, reverse=False):
    trie = Trie(reverse=reverse)
    for word in words:
        trie.insert(word)
    results = {}
    for word in words:
        stem, suffix = trie.find_split(word)
        results[word] = (stem, suffix)
    return results

## Comparing Prefix vs Suffix Tries

We compare the total suffix lengths to determine which trie gives better stem separation.

In [ ]:
# Comparison and Output
def compare_tries(filepath, output_file):
    words = load_words(filepath)

    with open(output_file, "w", encoding="utf-8") as out:
        out.write("Prefix Trie Stemming:\n")
        prefix_results = analyze_trie(words, reverse=False)
        for word, (stem, suffix) in prefix_results.items():
            out.write(f"{word} = {stem}+{suffix}\n")

        out.write("\nSuffix Trie Stemming:\n")
        suffix_results = analyze_trie(words, reverse=True)
        for word, (stem, suffix) in suffix_results.items():
            out.write(f"{word} = {stem}+{suffix}\n")

        prefix_suffix_total = sum(len(suffix) for _, suffix in prefix_results.values())
        suffix_suffix_total = sum(len(suffix) for _, suffix in suffix_results.values())

        out.write("\nComparison:\n")
        out.write(f"Total suffix length in Prefix Trie: {prefix_suffix_total}\n")
        out.write(f"Total suffix length in Suffix Trie: {suffix_suffix_total}\n")
        better = "Prefix Trie" if prefix_suffix_total < suffix_suffix_total else "Suffix Trie"
        out.write(f"Better stemming achieved by: {better}\n")

compare_tries("brown_nouns.txt", "stemming_output.txt")

# Reading and printing the output
with open("stemming_output.txt", "r", encoding="utf-8") as f:
    print(f.read())